In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install captcha
!pip install git+https://github.com/cocodataset/panopticapi.git
!git clone https://github.com/facebookresearch/detectron2.git
!pip install ./detectron2
!pip install --upgrade pyyaml

In [45]:
!streamlit run app.py &>/dev/null&

In [ ]:
%%writefile app.py

import re
import json
import requests
import streamlit as st
import cv2
from pyngrok import ngrok
from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt
import numpy as np
import random
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog

import string
characters = string.digits + string.ascii_uppercase

generator = ImageCaptcha(320, 100)

if 'img' not in st.session_state:
  st.session_state["random_str"] = ''.join([random.choice(characters) for i in range(6)])
  st.session_state["img"] = generator.generate_image(st.session_state["random_str"])

def main():
  html_temp = """
  <div>
    <div style ="width: 34%; margin-right: 1%; height: 100px; display: inline-block; float: left; background-image: url(https://www.santander.com.br/sites/WPC_CMS/imagem/22-08-16_175326_M_logo-santander.svg); background-repeat: no-repeat; background-position-x: center; background-position-y: center; background-size: 90%;">
    </div>
    <div style ="background: #CC0100; width: 65%; height: 100px; display: inline-block; padding: 23px;">
      <h3 style ="color: white; text-align: center;">Cadastre-se<h3>
    </div>
  </div>
  """

  # Função do stramlit que faz o display da webpage
  st.markdown(html_temp, unsafe_allow_html = True) 

  # As linhas abaixo criam as caixas na qual o usuário vai entrar com dados da pessoa que quer o empréstimo para fazer a Predição
  col1, col2 = st.columns(2)
  with col1:
    nome = st.text_input('Nome*:')

  with col2:
    sobrenome = st.text_input('Sobrenome*:')

  col3, col4 = st.columns(2)
  with col3:
    email = st.text_input('Email*:')

  with col4:
    telefone = st.text_input('Telefone*:')

  cpf = st.text_input('CPF*:')

  uploaded_file = st.file_uploader("Selecione uma foto sua segurando um objeto, ex: caneta, relógio, etc.*", help="A sua foto segurando um objeto é um método de segurança para as suas operações no nosso aplicativo")

  col5, col6 = st.columns(2)
  with col5:
    imagem = st.image(st.session_state["img"])

  with col6:
    textCaptcha = st.text_input("Captcha")

  def isEmailValid(email):
    regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
    return bool(re.fullmatch(regex, email))

  def validation(nome, sobrenome, email, telefone, cpf):
    flg_cpf_cnpj_recusado = False

    if st.button("Cadastrar"): 
      if nome and sobrenome and isEmailValid(email) and telefone.isnumeric() and cpf.isnumeric():
        if uploaded_file:
          if validateCaptcha(textCaptcha):
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, 1)
            if validateImage(image):
              st.success("Cadastro efetuado com sucesso!")
            else:
              st.error("Erro ao realizar o cadastro, perfil não autorizado!")
          else:
            st.warning('Captcha inválido, tente novamente!')  
        else:
          st.warning('Selecione uma foto sua segurando um objeto')  
      else:
        st.warning('Digite os campos corretamente')
        return

  def validateCaptcha(textCaptcha):
    return textCaptcha.upper() == st.session_state["random_str"]

  def validateImage(image):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)
    pred_classes = outputs['instances'].pred_classes.cpu().tolist()
    class_names = MetadataCatalog.get("coco_2017_val_panoptic_separated").thing_classes
    pred_class_names = list(map(lambda x: class_names[x], pred_classes))
    print(pred_class_names)
    try:
      return "person" in pred_class_names and "clock" in pred_class_names
    except Exception:
      return False

  validation(nome, sobrenome, email, telefone, cpf)

if __name__=='__main__': 
  main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501